In [ ]:
%load_ext rpy2.ipython

### Import packages

In [ ]:
%%R
library(ggplot2)
library(reshape2)
library(ggfortify)
library(ggplot2)
library(cluster) 
library(factoextra)
library(NbClust)

### Lets set the experiment WD and simulation WD

In [ ]:
%%R
exp_wd <- "Name"
sim_wd <- "Name"

### Inspect a Peak

In [ ]:
%%R
peak_data<-data.frame(read.csv(paste0("sim_wd","/","pattern_0/peak_0/data.csv")))
par(mfrow=c(10,1)) 
par(mar=c(1,1,1,1))
for (i in 0:9) {
    hist(subset(dat,modification==i)$position, xlim=c(0,3300), ylim = c(0, 100), breaks=200)
}

### Load the Data

In [ ]:
%%R
patterns <- list.dirs(path = mmd_wd, full.names = TRUE, recursive = FALSE)

for (pattern in patterns){
    pattern_name<-basename(pattern)
    dists_path<-list.files(path = pattern, full.names = TRUE, recursive = FALSE)[1]
    dists <- data.frame(read.csv(file=paste(dists_path), header=TRUE, sep=",")
}
